# FEATURE ENGINEERING STEPS
- Import data and packages
- Divide dataset into test and train
- Deal with missing values
- Encode and simplify categorical variables
- Log transformations for skewed numerical variables
- Scaling for numerical features

## <u>Import data and packages</u>

In [2]:
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
import numpy as np 
import json
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt




# import data
df_base = pd.read_csv('data/Base.csv') 

# import feature data types
with open('python_scripts/data_types.json') as f:
    data_types = json.load(f) 

# encoding source {'INTERNET', 'TELEAPP'} into source_is_internet_not_teleapp
df_base['source_is_internet_not_teleapp'] = (df_base['source'] == 'INTERNET').astype(int)
df_base.drop('source', axis=1, inplace=True)


# recording which variables have missing values recorded as -1
missing_values = ['prev_address_months_count',
                  'current_address_months_count',
                  'bank_months_count',
                  'session_length_in_minutes',
                  'device_distinct_emails_8w']

## <u>Divide dataset into training set and testing set</u>

Suggested splitting strategy by the creators of the datasets (Sérgio Jesus et al., Turning the Tables: Biased, Imbalanced, Dynamic Tabular Datasets for ML Evaluation, Feedzai, Universidade do Porto, INESCTEC.):

- "Splitting Strategy: Leveraging the “month” column, we were able to split the data temporally: the first six months for training, the last two for testing. This is common practice in the fraud domain — and the strategy used with the original dataset — as more recent data tends to be more faithful to the data’s distribution when models are put in production." (page 6)

My splitting strategy:
- Training set: first 6 months (model training)
- Validation set: 7th month (hyperparameter tuning/best model selection)
- Train + val set: first 7 months (for model refitting before testing on test set)
- Test set: 8th month (completely unseen data for final evaluation of the best model)

In [3]:
train = df_base[df_base.month <= 5].drop('month', axis=1)
val = df_base[df_base.month == 6].drop('month', axis=1)
test = df_base[df_base.month == 7].drop('month', axis=1)

X_train = train.drop('fraud_bool', axis=1)
y_train = train.fraud_bool

X_val = val.drop('fraud_bool', axis=1)
y_val = val.fraud_bool

X_train_val = pd.concat([X_train, X_val])
y_train_val = pd.concat([y_train, y_val])

X_test = test.drop('fraud_bool', axis=1)
y_test = test.fraud_bool

## <u>Deal with missing values</u>
#### Imputation strategy:
- Training set imputations will be calculated from only the train set
- Validation set imputations will also be calculated from only the train set
- Test set will be tested using a refitted model on the training and validation set combined, so imputations should be calculated based on the training and validation sets
- Combined training and validation sets will use the imputations calculated from these combined sets

#### FEATURE: device_distinct_emails_8w 
- impute missing values with mode (since it is categories, dont necessarily want middle value, just the most represented)


In [4]:
# device_distinct_emails_8w - impute missing values with mode (since it is categories, we want the most represented)

# training set only 
train_device_distinct_emails_8w_mode = X_train.device_distinct_emails_8w.value_counts().sort_values(ascending=False).index[0]
X_train.loc[X_train.device_distinct_emails_8w == -1, 'device_distinct_emails_8w'] = train_device_distinct_emails_8w_mode
X_val.loc[X_val.device_distinct_emails_8w == -1, 'device_distinct_emails_8w'] = train_device_distinct_emails_8w_mode

# training and validation set
train_val_device_distinct_emails_8w_mode = X_train_val.device_distinct_emails_8w.value_counts().sort_values(ascending=False).index[0]
X_train_val.loc[X_train_val.device_distinct_emails_8w == -1, 'device_distinct_emails_8w'] = train_val_device_distinct_emails_8w_mode
X_test.loc[X_test.device_distinct_emails_8w == -1, 'device_distinct_emails_8w'] = train_val_device_distinct_emails_8w_mode


#### FEATURE: session_length_in_minutes 
- impute missing values with median


In [5]:
# session_length_in_minutes - impute missing values with median

# training set only
train_session_length_in_minutes_median_position = int(np.ceil((len(X_train[X_train.session_length_in_minutes != -1])/2)-1))
train_session_length_in_minutes_median = sorted(X_train[X_train.session_length_in_minutes != -1].session_length_in_minutes)[train_session_length_in_minutes_median_position]
X_train.loc[X_train.session_length_in_minutes == -1, 'session_length_in_minutes'] = train_session_length_in_minutes_median
X_val.loc[X_val.session_length_in_minutes == -1, 'session_length_in_minutes'] = train_session_length_in_minutes_median

# training and validation set
train_val_session_length_in_minutes_median_position = int(np.ceil((len(X_train_val[X_train_val.session_length_in_minutes != -1])/2)-1))
train_val_session_length_in_minutes_median = sorted(X_train_val[X_train_val.session_length_in_minutes != -1].session_length_in_minutes)[train_val_session_length_in_minutes_median_position]
X_train_val.loc[X_train_val.session_length_in_minutes == -1, 'session_length_in_minutes'] = train_val_session_length_in_minutes_median
X_test.loc[X_test.session_length_in_minutes == -1, 'session_length_in_minutes'] = train_val_session_length_in_minutes_median


#### FEATURE: bank_months_count
- drop feature, create binary feature for value provided or not (more useful indicator)


In [6]:
# drop bank_months_count, create binary feature for value provided or not
X_train['bank_months_count_provided'] = [0 if i == -1 else 1 for i in X_train.bank_months_count]
X_train.drop('bank_months_count', axis=1, inplace=True)

X_val['bank_months_count_provided'] = [0 if i == -1 else 1 for i in X_val.bank_months_count]
X_val.drop('bank_months_count', axis=1, inplace=True)

X_train_val['bank_months_count_provided'] = [0 if i == -1 else 1 for i in X_train_val.bank_months_count]
X_train_val.drop('bank_months_count', axis=1, inplace=True)

X_test['bank_months_count_provided'] = [0 if i == -1 else 1 for i in X_test.bank_months_count]
X_test.drop('bank_months_count', axis=1, inplace=True)

#### FEATURE: prev_address_months_count
- drop feature (too many missing values), create binary feature for value provided or not


In [7]:
# drop prev_address_months_count (too many missing values), create binary feature for value provided or not
X_train['prev_address_months_count_provided'] = [0 if i == -1 else 1 for i in X_train.prev_address_months_count]
X_train.drop('prev_address_months_count', axis=1, inplace=True)

X_val['prev_address_months_count_provided'] = [0 if i == -1 else 1 for i in X_val.prev_address_months_count]
X_val.drop('prev_address_months_count', axis=1, inplace=True)

X_train_val['prev_address_months_count_provided'] = [0 if i == -1 else 1 for i in X_train_val.prev_address_months_count]
X_train_val.drop('prev_address_months_count', axis=1, inplace=True)

X_test['prev_address_months_count_provided'] = [0 if i == -1 else 1 for i in X_test.prev_address_months_count]
X_test.drop('prev_address_months_count', axis=1, inplace=True)

#### FEATURE: current_address_months_count 
- impute missing values with median


In [8]:
# current_address_months_count - impute missing values with median
# and create binary features for value provided or not

X_train['current_address_months_count_provided'] = [0 if i == -1 else 1 for i in X_train.current_address_months_count]
X_val['current_address_months_count_provided'] = [0 if i == -1 else 1 for i in X_val.current_address_months_count]
X_train_val['current_address_months_count_provided'] = [0 if i == -1 else 1 for i in X_train_val.current_address_months_count]
X_test['current_address_months_count_provided'] = [0 if i == -1 else 1 for i in X_test.current_address_months_count]

# training set only
train_current_address_months_count_median_position = int(np.ceil((len(X_train[X_train['current_address_months_count'] != -1])/2)-1))
train_current_address_months_count_median = sorted(X_train[X_train['current_address_months_count'] != -1]['current_address_months_count'])[train_current_address_months_count_median_position]
X_train.loc[X_train['current_address_months_count'] == -1, 'current_address_months_count'] = train_current_address_months_count_median
X_val.loc[X_val['current_address_months_count'] == -1, 'current_address_months_count'] = train_current_address_months_count_median

# training and validation set
train_val_current_address_months_count_median_position = int(np.ceil((len(X_train[X_train['current_address_months_count'] != -1])/2)-1))
train_val_current_address_months_count_median = sorted(X_train[X_train['current_address_months_count'] != -1]['current_address_months_count'])[train_val_current_address_months_count_median_position]
X_train_val.loc[X_train_val['current_address_months_count'] == -1, 'current_address_months_count'] = train_val_current_address_months_count_median
X_test.loc[X_test['current_address_months_count'] == -1, 'current_address_months_count'] = train_val_current_address_months_count_median

## <u>Encode and simplfy categorical features</u>

In [9]:
# drop device_fraud_count (single value)
X_train.drop('device_fraud_count', axis=1, inplace=True)
X_val.drop('device_fraud_count', axis=1, inplace=True)
X_train_val.drop('device_fraud_count', axis=1, inplace=True)
X_test.drop('device_fraud_count', axis=1, inplace=True)

In [10]:
# simplying proposed credit limit to 5 categories
train_categories_to_keep = X_train.proposed_credit_limit.value_counts().index[:4] # taking 4 largest represented proposed limits
X_train['proposed_credit_limit'] = X_train['proposed_credit_limit'].where(X_train['proposed_credit_limit'].isin(train_categories_to_keep), 'other').astype(str)
X_val['proposed_credit_limit'] = X_val['proposed_credit_limit'].where(X_val['proposed_credit_limit'].isin(train_categories_to_keep), 'other').astype(str)

train_val_categories_to_keep = X_train_val.proposed_credit_limit.value_counts().index[:4] # taking 4 largest represented proposed limits
X_train_val['proposed_credit_limit'] = X_train_val['proposed_credit_limit'].where(X_train_val['proposed_credit_limit'].isin(train_val_categories_to_keep), 'other').astype(str)
X_test['proposed_credit_limit'] = X_test['proposed_credit_limit'].where(X_test['proposed_credit_limit'].isin(train_val_categories_to_keep), 'other').astype(str)

In [11]:
# one hot encoding
categories_to_encode = ['proposed_credit_limit', 'payment_type', 'employment_status', 'housing_status', 'device_os']

# on train set
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first') 
encoded_array_train = ohe.fit_transform(X_train[categories_to_encode])
encoded_df = pd.DataFrame(encoded_array_train, columns=ohe.get_feature_names_out())
X_train_encoded = pd.concat([X_train.drop(columns=categories_to_encode).reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)

encoded_array_val = ohe.transform(X_val[categories_to_encode])
encoded_df_val = pd.DataFrame(encoded_array_val, columns=ohe.get_feature_names_out())
X_val_encoded = pd.concat([X_val.drop(columns=categories_to_encode).reset_index(drop=True), encoded_df_val.reset_index(drop=True)], axis=1)


# on train and val set
ohe_train_val = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first') 
encoded_array_train_val = ohe_train_val.fit_transform(X_train_val[categories_to_encode])
encoded_df = pd.DataFrame(encoded_array_train_val, columns=ohe_train_val.get_feature_names_out())
X_train_val_encoded = pd.concat([X_train_val.drop(columns=categories_to_encode).reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)


encoded_array_test = ohe_train_val.transform(X_test[categories_to_encode])
encoded_df_test = pd.DataFrame(encoded_array_test, columns=ohe_train_val.get_feature_names_out())
X_test_encoded = pd.concat([X_test.drop(columns=categories_to_encode).reset_index(drop=True), encoded_df_test.reset_index(drop=True)], axis=1)


In [12]:
X_train_encoded

,income,name_email_similarity,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,zip_count_4w,velocity_6h,velocity_24h,velocity_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,credit_risk_score,email_is_free,phone_home_valid,phone_mobile_valid,has_other_cards,foreign_request,session_length_in_minutes,keep_alive_session,device_distinct_emails_8w,source_is_internet_not_teleapp,bank_months_count_provided,prev_address_months_count_provided,current_address_months_count_provided,proposed_credit_limit_1500.0,proposed_credit_limit_200.0,proposed_credit_limit_500.0,proposed_credit_limit_other,payment_type_AB,payment_type_AC,payment_type_AD,payment_type_AE,employment_status_CB,employment_status_CC,employment_status_CD,employment_status_CE,employment_status_CF,employment_status_CG,housing_status_BB,housing_status_BC,housing_status_BD,housing_status_BE,housing_status_BF,housing_status_BG,device_os_macintosh,device_os_other,device_os_windows,device_os_x11
0,0.3,0.986506,25,40,0.006735,102.453711,1059,13096.035018,7850.955007,6742.080561,5,5,163,1,0,1,0,0,16.224843,1,1,1,1,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.8,0.617426,89,20,0.010095,-0.849551,1658,9223.283431,5745.251481,5941.664859,3,18,154,1,1,1,0,0,3.363854,1,1,1,1,0,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.8,0.996707,14,40,0.012316,-1.490386,1095,4471.472149,5471.988958,5992.555113,15,11,89,1,0,1,0,0,22.730559,0,1,1,1,1,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.6,0.475100,14,30,0.006991,-1.863101,3483,14431.993621,6755.344479,5970.336831,11,13,90,1,0,1,0,0,15.215816,1,1,1,1,1,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.9,0.842307,29,40,5.742626,47.152498,2339,7601.511579,5124.046930,5940.734212,1,6,91,0,1,1,0,0,3.743048,0,1,1,1,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794984,0.9,0.670058,100,20,0.031334,-0.285938,2674,3445.858573,4733.316718,4371.352072,2,8,247,1,0,1,0,0,0.115263,0,1,1,1,0,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
794985,0.6,0.369424,15,30,0.016822,-0.105671,804,5361.336674,5022.548146,4887.068435,13,19,176,1,0,1,0,0,8.441716,1,1,1,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
794986,0.6,0.245233,40,20,0.012318,-0.710525,1165,7011.340079,3736.164978,4203.504843,14,15,30,1,0,1,0,0,0.548392,0,1,1,1,0,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
794987,0.1,0.653664,9,20,0.008505,-0.446204,2971,3347.956800,3570.438099,4252.645154,0,10,42,1,1,1,0,0,5.565154,0,1,1,0,0,1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
X_val_encoded

,income,name_email_similarity,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,zip_count_4w,velocity_6h,velocity_24h,velocity_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,credit_risk_score,email_is_free,phone_home_valid,phone_mobile_valid,has_other_cards,foreign_request,session_length_in_minutes,keep_alive_session,device_distinct_emails_8w,source_is_internet_not_teleapp,bank_months_count_provided,prev_address_months_count_provided,current_address_months_count_provided,proposed_credit_limit_1500.0,proposed_credit_limit_200.0,proposed_credit_limit_500.0,proposed_credit_limit_other,payment_type_AB,payment_type_AC,payment_type_AD,payment_type_AE,employment_status_CB,employment_status_CC,employment_status_CD,employment_status_CE,employment_status_CF,employment_status_CG,housing_status_BB,housing_status_BC,housing_status_BD,housing_status_BE,housing_status_BF,housing_status_BG,device_os_macintosh,device_os_other,device_os_windows,device_os_x11
0,0.9,0.536292,129,50,0.013297,36.472905,1360,3584.044957,2110.445975,3536.218324,9,5,250,0,1,1,1,0,1.842079,0,1,1,1,0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.2,0.592649,199,20,0.000148,-1.472056,1953,2424.702941,2804.784273,3504.418475,1,9,-35,0,0,1,0,0,7.976496,1,1,1,0,0,1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.6,0.381600,18,30,0.763345,-1.875981,889,5783.400951,3942.258389,4171.934488,1384,6,190,1,0,1,0,0,9.500332,1,1,1,1,0,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.3,0.771565,112,40,0.008299,-0.812164,1127,2466.877754,2828.730243,4333.512699,16,4,295,1,1,1,0,0,4.216191,0,1,1,1,0,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.1,0.915050,116,20,0.030500,-1.100876,1748,6108.017018,4597.491949,4277.897700,1,8,-57,1,1,1,0,0,5.500443,1,1,1,0,0,1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108163,0.4,0.171547,282,60,0.873605,-1.339224,892,2673.871092,5915.266603,4353.516751,0,1,145,1,0,1,0,0,7.008432,0,1,1,0,0,1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
108164,0.3,0.363037,44,30,0.006526,-1.470696,1447,2850.820595,3003.996773,4407.067580,11,10,148,0,1,1,0,0,1.878316,1,1,1,1,0,1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
108165,0.9,0.689170,16,40,0.004897,-0.887880,460,1828.525055,5007.106989,4284.321173,1,10,241,0,0,1,0,0,3.731549,1,1,1,0,1,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
108166,0.9,0.670834,62,40,0.001467,31.374154,837,1914.866253,6433.236446,4029.823350,25,4,66,0,1,0,0,0,37.570273,0,1,1,1,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [14]:
X_train_val_encoded

,income,name_email_similarity,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,zip_count_4w,velocity_6h,velocity_24h,velocity_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,credit_risk_score,email_is_free,phone_home_valid,phone_mobile_valid,has_other_cards,foreign_request,session_length_in_minutes,keep_alive_session,device_distinct_emails_8w,source_is_internet_not_teleapp,bank_months_count_provided,prev_address_months_count_provided,current_address_months_count_provided,proposed_credit_limit_1500.0,proposed_credit_limit_200.0,proposed_credit_limit_500.0,proposed_credit_limit_other,payment_type_AB,payment_type_AC,payment_type_AD,payment_type_AE,employment_status_CB,employment_status_CC,employment_status_CD,employment_status_CE,employment_status_CF,employment_status_CG,housing_status_BB,housing_status_BC,housing_status_BD,housing_status_BE,housing_status_BF,housing_status_BG,device_os_macintosh,device_os_other,device_os_windows,device_os_x11
0,0.3,0.986506,25,40,0.006735,102.453711,1059,13096.035018,7850.955007,6742.080561,5,5,163,1,0,1,0,0,16.224843,1,1,1,1,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.8,0.617426,89,20,0.010095,-0.849551,1658,9223.283431,5745.251481,5941.664859,3,18,154,1,1,1,0,0,3.363854,1,1,1,1,0,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.8,0.996707,14,40,0.012316,-1.490386,1095,4471.472149,5471.988958,5992.555113,15,11,89,1,0,1,0,0,22.730559,0,1,1,1,1,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.6,0.475100,14,30,0.006991,-1.863101,3483,14431.993621,6755.344479,5970.336831,11,13,90,1,0,1,0,0,15.215816,1,1,1,1,1,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.9,0.842307,29,40,5.742626,47.152498,2339,7601.511579,5124.046930,5940.734212,1,6,91,0,1,1,0,0,3.743048,0,1,1,1,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903152,0.4,0.171547,282,60,0.873605,-1.339224,892,2673.871092,5915.266603,4353.516751,0,1,145,1,0,1,0,0,7.008432,0,1,1,0,0,1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
903153,0.3,0.363037,44,30,0.006526,-1.470696,1447,2850.820595,3003.996773,4407.067580,11,10,148,0,1,1,0,0,1.878316,1,1,1,1,0,1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
903154,0.9,0.689170,16,40,0.004897,-0.887880,460,1828.525055,5007.106989,4284.321173,1,10,241,0,0,1,0,0,3.731549,1,1,1,0,1,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
903155,0.9,0.670834,62,40,0.001467,31.374154,837,1914.866253,6433.236446,4029.823350,25,4,66,0,1,0,0,0,37.570273,0,1,1,1,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [15]:
X_test_encoded

,income,name_email_similarity,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,zip_count_4w,velocity_6h,velocity_24h,velocity_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,credit_risk_score,email_is_free,phone_home_valid,phone_mobile_valid,has_other_cards,foreign_request,session_length_in_minutes,keep_alive_session,device_distinct_emails_8w,source_is_internet_not_teleapp,bank_months_count_provided,prev_address_months_count_provided,current_address_months_count_provided,proposed_credit_limit_1500.0,proposed_credit_limit_200.0,proposed_credit_limit_500.0,proposed_credit_limit_other,payment_type_AB,payment_type_AC,payment_type_AD,payment_type_AE,employment_status_CB,employment_status_CC,employment_status_CD,employment_status_CE,employment_status_CF,employment_status_CG,housing_status_BB,housing_status_BC,housing_status_BD,housing_status_BE,housing_status_BF,housing_status_BG,device_os_macintosh,device_os_other,device_os_windows,device_os_x11
0,0.9,0.631260,35,40,0.012314,-1.334990,775,3255.908260,2391.592313,3197.046015,0,4,169,0,1,1,1,0,3.424854,1,1,1,0,0,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.9,0.890712,45,30,0.003810,-1.432133,382,4213.456672,2763.853399,3076.483662,8,2,201,1,1,1,1,0,2.020729,0,1,1,1,0,1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.3,0.228200,242,40,0.038756,-1.476443,1091,3431.573962,2221.336130,3166.003104,299,4,107,0,1,1,0,0,3.870224,0,1,1,1,0,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.3,0.488990,34,30,0.023142,-1.339456,1092,1867.026264,2608.458509,3116.152551,14,10,186,1,0,1,0,0,16.633363,1,1,1,1,0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.5,0.654936,262,20,0.024962,-0.602520,3591,1317.378569,6207.504542,3090.810224,9,7,203,0,1,1,1,0,5.309040,0,1,1,1,0,1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96838,0.8,0.124690,143,30,0.051348,-0.826239,530,6732.602414,3010.048099,3095.754245,42,8,305,1,1,1,0,0,16.967770,0,1,1,1,0,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
96839,0.9,0.824544,193,30,0.009591,0.008307,408,1574.293294,2716.495767,4286.089050,0,5,235,0,1,1,1,0,1.504109,0,1,1,0,0,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
96840,0.8,0.140891,202,10,0.059287,50.609995,749,1258.864938,3601.322892,3103.891664,2,3,195,1,0,1,0,0,16.068595,0,1,1,1,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
96841,0.9,0.002480,3,30,0.023357,-1.313387,707,7048.137128,6521.395012,3068.265084,7,8,148,0,0,1,0,0,1.378683,1,1,1,1,1,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## <u>Log transformations of skewed features</u>

In [16]:
# test for skewness for possible log transformation
from scipy.stats import skew
numericals = data_types['numerical_continuous_bounded']+data_types['numerical_continuous_unbounded']+data_types['numerical_discrete']+['customer_age']
to_drop = ['prev_address_months_count', 'bank_months_count'] # no longer in training set
features = []
skews = []
min_values = []
for i in numericals:
    if i not in to_drop:
        features.append(i)
        skews.append(skew(X_train_encoded[i]))
        min_values.append(min(X_train_encoded[i]))
skews_df = pd.DataFrame({'Feature':features, 'Skewness':skews, 'Min':min_values}).sort_values('Skewness', ascending=False).set_index('Feature')
skews_df.drop('Min', axis=1)

,Skewness
Feature,
days_since_request,9.120598
session_length_in_minutes,3.267051
bank_branch_count_8w,2.693416
intended_balcon_amount,2.463442
current_address_months_count,1.389794
zip_count_4w,1.375867
date_of_birth_distinct_emails_4w,0.601809
customer_age,0.491786
velocity_6h,0.407529


In [17]:
significant_skews = list(skews_df[skews_df.Skewness >= 1].index)
# shift variables to push negative values above 0
shifted_df = {}
for i in significant_skews:
    min_value = skews_df.loc[i].Min 
    if min_value <1 :
        shifted_df[i] = X_train_encoded[i] + 1 + abs(min_value)
    else:
        shifted_df[i] = X_train_encoded[i]

# check if range of values spans 3 orders of magnitude (largest value is over 10^3 time the smallest value)
features = []
range_orders_of_magnitude = []
for i,j in shifted_df.items():
    log_min = np.log10(min(j))
    log_max = np.log10(max(j))
    range_orders_of_magnitude.append(log_max - log_min)
    features.append(i)
magnitude_of_ranges_df = pd.DataFrame({'Feature': features, 
                                       'range_orders_of_magnitude':range_orders_of_magnitude}).sort_values('range_orders_of_magnitude', ascending=False).set_index('Feature')
magnitude_of_ranges_df = magnitude_of_ranges_df.join(skews_df.drop('Min', axis=1), on='Feature', how='left')
magnitude_of_ranges_df

,range_orders_of_magnitude,Skewness
Feature,,
zip_count_4w,3.826075,1.375867
bank_branch_count_8w,3.376942,2.693416
current_address_months_count,2.629410,1.389794
intended_balcon_amount,2.112228,2.463442
session_length_in_minutes,1.936207,3.267051
days_since_request,1.889758,9.120598


- `current_address_months_count` displays moderate skew and range below 3 orders of magnitude, so I will not log transform this variable.
- Log transformation will be performed on the rest of the features displayed 


### Important Consideration: what if there are values in the test set smaller than the minimum value of the training set?
- These could potentially remain below 0, this would break the log transformation function
    - solution: set negatives to 0

In [18]:
# executing log transformations on train and val sets
to_transform = list(magnitude_of_ranges_df.index.drop(['current_address_months_count']))

# the features will be shifted up by the following values before log transformation occurs (if minimum value of a feature is 1 or above, there will be no shift)
feature_shifts = {}
for i in to_transform:
    min_value = min(X_train[i])
    if min_value < 1:
        feature_shifts[i] = 1 + abs(min_value)
    else:
        feature_shifts[i] = 0

X_train_encoded_transformed = X_train_encoded.copy()
X_val_encoded_transformed = X_val_encoded.copy()

# log transformations 
for i in to_transform:
    # on training set
    X_train_encoded_transformed[i] = np.log(X_train_encoded[i] + feature_shifts[i])

    # on test set using feature shifts defined by training set
    if feature_shifts[i] == 0:
        val_pre_log = list(map(lambda x: 0 if x < 0 else x, X_val_encoded[i]))
    else:
        val_pre_log = X_val_encoded[i] + feature_shifts[i]
        val_pre_log = list(map(lambda x: feature_shifts[i] if x < feature_shifts[i] else x, val_pre_log))
    X_val_encoded_transformed[i] = np.log(val_pre_log)

In [19]:
# executing log transformations on train_val and test sets
feature_shifts = {}
for i in to_transform:
    min_value = min(X_train_val[i])
    if min_value < 1:
        feature_shifts[i] = 1 + abs(min_value)
    else:
        feature_shifts[i] = 0

X_train_val_encoded_transformed = X_train_val_encoded.copy()
X_test_encoded_transformed = X_test_encoded.copy()

# log transformations 
for i in to_transform:
    # on training set
    X_train_val_encoded_transformed[i] = np.log(X_train_val_encoded[i] + feature_shifts[i])

    # on test set using feature shifts defined by training set
    if feature_shifts[i] == 0:
        test_pre_log = list(map(lambda x: 0 if x < 0 else x, X_test_encoded[i]))
    else:
        test_pre_log = X_test_encoded[i] + feature_shifts[i]
        test_pre_log = list(map(lambda x: feature_shifts[i] if x < feature_shifts[i] else x, test_pre_log))
    X_test_encoded_transformed[i] = np.log(test_pre_log)

In [20]:
X_train[to_transform]

,zip_count_4w,bank_branch_count_8w,intended_balcon_amount,session_length_in_minutes,days_since_request
0,1059,5,102.453711,16.224843,0.006735
1,1658,3,-0.849551,3.363854,0.010095
2,1095,15,-1.490386,22.730559,0.012316
3,3483,11,-1.863101,15.215816,0.006991
4,2339,1,47.152498,3.743048,5.742626
...,...,...,...,...,...
794984,2674,2,-0.285938,0.115263,0.031334
794985,804,13,-0.105671,8.441716,0.016822
794986,1165,14,-0.710525,0.548392,0.012318
794987,2971,0,-0.446204,5.565154,0.008505


In [21]:
X_train_encoded_transformed[to_transform]


,zip_count_4w,bank_branch_count_8w,intended_balcon_amount,session_length_in_minutes,days_since_request
0,6.965080,1.791759,4.778991,2.846542,0.006713
1,7.413367,1.386294,2.752450,1.474103,0.010044
2,6.998510,2.772589,2.710725,3.166901,0.012241
3,8.155649,2.484907,2.685631,2.786188,0.006967
4,7.757479,0.693147,4.153918,1.557368,1.908449
...,...,...,...,...,...
794984,7.891331,1.098612,2.787762,0.112012,0.030853
794985,6.689599,2.639057,2.798798,2.245483,0.016682
794986,7.060476,2.708050,2.761277,0.439322,0.012243
794987,7.996654,0.000000,2.777847,1.882273,0.008469


In [22]:
# compare skewness after transformation
features = []
skews = []
for i in to_transform:
        features.append(i)
        skews.append(skew(X_train_encoded_transformed[i]))
skews_df = pd.DataFrame({'Feature':features, 'Skewness_After':skews}).set_index('Feature')
magnitude_of_ranges_df.join(skews_df, on='Feature', how='inner').drop('range_orders_of_magnitude', axis=1).rename(columns={'Skewness':'Skewness_Before'})

,Skewness_Before,Skewness_After
Feature,,
zip_count_4w,1.375867,-0.244096
bank_branch_count_8w,2.693416,0.883013
intended_balcon_amount,2.463442,1.530833
session_length_in_minutes,3.267051,0.505004
days_since_request,9.120598,3.654814


## <u>Scaling for numerical features</u>

In [23]:
# select features for scaling
numericals = data_types['numerical_continuous_bounded']+data_types['numerical_continuous_unbounded']+data_types['numerical_discrete']+data_types['ordinal']+data_types['temporal']
to_drop = ['prev_address_months_count', 'bank_months_count', 'month'] # variables not used anymore
to_scale = [i for i in numericals if i not in to_drop]

Key points:
- Robust Scaler for features with lower orders of magnitude
- Power Transformer for features with higher orders of magnitude

In [24]:
from sklearn.preprocessing import RobustScaler, PowerTransformer

In [25]:
# test for orders of magnitude
# check if range of values spans 3 orders of magnitude (largest value is over 10^3 time the smallest value)
features = []
range_orders_of_magnitude = []
for i in to_scale:
    if X_train_encoded_transformed[i].min() > 0:
        log_min = np.log10(X_train_encoded_transformed[i].min())
        log_max = np.log10(X_train_encoded_transformed[i].max())
        range_orders_of_magnitude.append(log_max - log_min)
        features.append(i)
magnitude_of_ranges_df = pd.DataFrame({'Feature': features, 
                                       'range_orders_of_magnitude':range_orders_of_magnitude}).sort_values('range_orders_of_magnitude', ascending=False).set_index('Feature')
magnitude_of_ranges_df


,range_orders_of_magnitude
Feature,
days_since_request,8.731548
name_email_similarity,5.843284
session_length_in_minutes,2.836653
customer_age,0.954243
income,0.954243
velocity_24h,0.855893
velocity_4w,0.387833


In [26]:
# SCALING FOR TRAIN AND VAL SETS

# for power transformer
to_scale_pt = magnitude_of_ranges_df[magnitude_of_ranges_df.range_orders_of_magnitude >= 3].index
# for robust scaler
to_scale_rs = [i for i in to_scale if i not in to_scale_pt]

# copy of datasets
X_train_encoded_transformed_scaled = X_train_encoded_transformed.copy()
X_val_encoded_transformed_scaled = X_val_encoded_transformed.copy()

# apply robust scaling
rs = RobustScaler()

X_train_encoded_transformed_scaled[to_scale_rs] = rs.fit_transform(X_train_encoded_transformed[to_scale_rs])
X_val_encoded_transformed_scaled[to_scale_rs] = rs.transform(X_val_encoded_transformed[to_scale_rs])

# apply power transforming
pt = PowerTransformer()

X_train_encoded_transformed_scaled[to_scale_pt] = pt.fit_transform(X_train_encoded_transformed[to_scale_pt])
X_val_encoded_transformed_scaled[to_scale_pt] = pt.transform(X_val_encoded_transformed[to_scale_pt])

In [27]:
# SCALING FOR TRAIN_VAL AND TEST SETS

# copy of datasets
X_train_val_encoded_transformed_scaled = X_train_val_encoded_transformed.copy()
X_test_encoded_transformed_scaled = X_test_encoded_transformed.copy()

# apply robust scaling
rs = RobustScaler()

X_train_val_encoded_transformed_scaled[to_scale_rs] = rs.fit_transform(X_train_val_encoded_transformed[to_scale_rs])
X_test_encoded_transformed_scaled[to_scale_rs] = rs.transform(X_test_encoded_transformed[to_scale_rs])

# apply power transforming
pt = PowerTransformer()

X_train_val_encoded_transformed_scaled[to_scale_pt] = pt.fit_transform(X_train_val_encoded_transformed[to_scale_pt])
X_test_encoded_transformed_scaled[to_scale_pt] = pt.transform(X_test_encoded_transformed[to_scale_pt])

In [28]:
X_train_encoded_transformed_scaled[to_scale]

,name_email_similarity,days_since_request,intended_balcon_amount,velocity_6h,velocity_24h,velocity_4w,session_length_in_minutes,current_address_months_count,bank_branch_count_8w,date_of_birth_distinct_emails_4w,credit_risk_score,zip_count_4w,income,customer_age
0,1.623692,-0.556545,4.462747,1.722505,1.514336,1.491359,1.190744,-0.252252,-0.223827,-0.571429,0.438776,-0.323694,-0.6,0.5
1,0.431571,-0.457336,-0.004311,0.787539,0.387789,0.756553,-0.418667,0.324324,-0.373553,1.285714,0.346939,0.254667,0.4,-0.5
2,1.655585,-0.393673,-0.096285,-0.359650,0.241594,0.803271,1.566418,-0.351351,0.138363,0.285714,-0.316327,-0.280565,0.4,0.5
3,-0.050765,-0.548876,-0.151598,2.045034,0.928187,0.782874,1.119969,-0.351351,0.032131,0.571429,-0.306122,1.212328,0.0,0.0
4,1.167095,2.874455,3.084914,0.396009,0.055446,0.755698,-0.321026,-0.216216,-0.629511,-0.428571,-0.295918,0.698625,0.6,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794984,0.606485,0.094564,0.073526,-0.607255,-0.153594,-0.685044,-2.015944,0.423423,-0.479785,-0.142857,1.295918,0.871315,0.6,-0.5
794985,-0.418517,-0.269051,0.097852,-0.144818,0.001144,-0.211600,0.485903,-0.342342,0.089054,1.428571,0.571429,-0.679108,0.0,0.0
794986,-0.862456,-0.393629,0.015146,0.253529,-0.687068,-0.839133,-1.632119,-0.117117,0.114531,0.857143,-0.918367,-0.200618,0.0,-0.5
794987,0.552190,-0.503845,0.051671,-0.630891,-0.775732,-0.794021,0.059979,-0.396396,-0.885469,0.142857,-0.795918,1.007199,-1.0,-0.5


In [33]:
import os
data_folder = os.path.join(os.getcwd(), 'data')

# X_train
file_path = os.path.join(data_folder, 'X_train.parquet')
X_train_encoded_transformed_scaled.to_parquet(file_path)

# X_val
file_path = os.path.join(data_folder, 'X_val.parquet')
X_val_encoded_transformed_scaled.to_parquet(file_path)

# X_train_val
file_path = os.path.join(data_folder, 'X_train_val.parquet')
X_train_val_encoded_transformed_scaled.to_parquet(file_path)

# X_test
file_path = os.path.join(data_folder, 'X_test.parquet')
X_test_encoded_transformed_scaled.to_parquet(file_path)



# y_train
file_path = os.path.join(data_folder, 'y_train.csv')
y_train.to_csv(file_path, index=False)

# y_val
file_path = os.path.join(data_folder, 'y_val.csv')
y_val.to_csv(file_path, index=False)

# y_train
file_path = os.path.join(data_folder, 'y_train_val.csv')
y_train_val.to_csv(file_path, index=False)

# y_test
file_path = os.path.join(data_folder, 'y_test.csv')
y_test.to_csv(file_path, index=False)